In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf
import string
import re

In [3]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select \
                  x.q_id \
                , x.q_posttypeid \
                , x.q_acceptedanswerid \
                , x.q_parentid \
                , x.q_creationdate \
                , x.q_score \
                , x.q_viewcount \
                , x.q_owneruserid \
                , x.q_title \
                , x.q_tags \
                , x.q_answercount \
                , x.q_commentcount \
                , x.q_favoritecount \
                , x.q_closeddate \
                , b.id               as a_id \
                , b.posttypeid       as a_posttypeid \
                , b.acceptedanswerid as a_acceptedanswerid \
                , b.parentid         as a_parentid \
                , b.creationdate     as a_creationdate \
                , b.score            as a_score \
                , b.viewcount        as a_viewcount \
                , b.owneruserid      as a_owneruserid \
                , b.title            as a_title \
                , b.tags             as a_tags \
                , b.answercount      as a_answercount \
                , b.commentcount     as a_commentcount \
                , b.favoritecount    as a_favoritecount \
                , b.closeddate       as a_closeddate \
  from ( \
           select a.id               as q_id \
                , a.posttypeid       as q_posttypeid \
                , a.acceptedanswerid as q_acceptedanswerid \
                , a.parentid         as q_parentid \
                , a.creationdate     as q_creationdate \
                , a.score            as q_score \
                , a.viewcount        as q_viewcount \
                , a.owneruserid      as q_owneruserid \
                , a.title            as q_title \
                , a.answercount      as q_answercount \
                , a.commentcount     as q_commentcount \
                , a.favoritecount    as q_favoritecount \
                , a.closeddate       as q_closeddate \
                , replace(replace(lower(a.tags), '<', ''), '>', ' ')as q_tags \
           from posts a \
           where a.creationdate between '2013-01-01' and '2023-12-01' \
             and a.posttypeid = '1' \
             and a.owneruserid is not null \
       )   x \
        , posts b \
where b.parentid = x.q_id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [4]:
df_da = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_posttypeid' 
, 'q_acceptedanswerid'
, 'q_parentid' 
, 'q_creationdate' 
, 'q_score' 
, 'q_viewcount' 
, 'q_owneruserid' 
, 'q_title' 
, 'q_tags' 
, 'q_answercount' 
, 'q_commentcount' 
, 'q_favoritecount'
, 'q_closeddate' 
, 'a_id' 
, 'a_posttypeid' 
, 'a_acceptedanswerid' 
, 'a_parentid' 
, 'a_creationdate' 
, 'a_score' 
, 'a_viewcount' 
, 'a_owneruserid' 
, 'a_title' 
, 'a_tags' 
, 'a_answercount'
, 'a_commentcount'
, 'a_favoritecount'
, 'a_closeddate' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_da.pkl', 'wb') as f:
    pickle.dump(df_da, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select a.creationdate \
     , a.id \
     , a.posttypeid \
     , a.title \
     , a.owneruserid \
     , b.body \
     , case when c.id is null then 0 else 1 end as answered_yn \
  from posts as a \
     inner join postsbody b \
     on a.id = b.id \
     left outer join vw_posts_get_answer as c \
     on a.id = c.id \
where a.creationdate between '2013-11-01' and '2023-12-01' \
  ; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_body = pd.DataFrame(rows, columns = [
  'creationdate' 
, 'id' 
, 'posttypeid'
, 'title'
, 'owneruserid'
, 'body' 
, 'answered_yn'
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_body.pkl', 'wb') as f:
    pickle.dump(df_body, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select id, creationdate, displayname, lastaccessdate \
  from users \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_user = pd.DataFrame(rows, columns = [
  'id' 
, 'creationdate' 
, 'displayname'
, 'lastaccessdate'
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_user.pkl', 'wb') as f:
    pickle.dump(df_user, f)

In [4]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select to_char(p.creationdate, 'yyyy') as year, p.id, count(a.id) as cnt \
  from posts p \
     left join posts a \
     on p.id = a.parentid \
     and a.posttypeid = '2' \
     and a.creationdate - p.creationdate <= '30 days' \
 where ((p.creationdate between '2023-11-01' and '2023-12-01') \
 	 or (p.creationdate between '2022-11-01' and '2022-12-01') \
 	 or (p.creationdate between '2021-11-01' and '2021-12-01') \
 	 or (p.creationdate between '2020-11-01' and '2020-12-01') \
 	 or (p.creationdate between '2019-11-01' and '2019-12-01') \
 	 ) \
   and p.posttypeid ='1' \
  group by p.id \
  ; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [5]:
df_answer_cnt = pd.DataFrame(rows, columns = [
  'year' 
, 'id'
, 'cnt' 
])

In [6]:
# 전체 데이터셋 
with open('../../data/df_answer_cnt.pkl', 'wb') as f:
    pickle.dump(df_answer_cnt, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select xx.year, xx.usr_id, count(xx.q_id) as cnt \
  from ( \
 select x.usr_id, x.usr_creationdate, x.year, p.id as q_id, p.creationdate as q_date \
   from ( \
  	select u.id as usr_id, u.creationdate as usr_creationdate, m.year \
     from users u \
        , ma_year m \
 	where to_char(u.creationdate, 'yyyy') <= m.year \
   	  and m.year between '2013' and '2023' \
      and u.id >1 \
  	) x \
   left join posts p \
   on x.usr_id = p.owneruserid \
   and p.posttypeid = '1' \
   and x.year = to_char(p.creationdate, 'yyyy') \
) xx \
group by xx.year, xx.usr_id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_usr_q_cnt = pd.DataFrame(rows, columns = [
  'year' 
, 'usr_id'
, 'cnt' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_usr_q_cnt.pkl', 'wb') as f:
    pickle.dump(df_usr_q_cnt, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select xxx.year, avg(xxx.cnt) as avg cnt \
  from ( \
		  select xx.year, xx.usr_id, count(xx.q_id) as cnt \
		  from (select x.usr_id, x.usr_creationdate, x.year, p.id as q_id, p.creationdate as q_date \
		   from ( \
		  	select u.id as usr_id, u.creationdate as usr_creationdate, m.year \
		     from users u \
		        , ma_year m \
		 	where to_char(u.creationdate, 'yyyy') <= m.year \
		   	  and m.year between '2013' and '2023' \
		      and u.id >1 \
		  	) x \
		   left join posts p \
		   on x.usr_id = p.owneruserid \
		   and p.posttypeid = '2' \
		   and x.year = to_char(p.creationdate, 'yyyy') \
		) xx \
		group by xx.year, xx.usr_id \
  ) xxx \
 group by xxx.year \
  ;              "
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_usr_a_cnt = pd.DataFrame(rows, columns = [
  'year' 
, 'usr_id'
, 'cnt' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_usr_a_cnt.pkl', 'wb') as f:
    pickle.dump(df_usr_a_cnt, f)

In [ ]:
# create table tt_year_from_signup_qna as
# select x.usr_id
#      , p.id as q_id
#      , ceil(DATE_PART('day',p.creationdate::timestamp-x.usr_creationdate::timestamp)/365) as yeardiff
#      , p.creationdate, x.usr_creationdate
#   from (
#         select u.id as usr_id, u.creationdate as usr_creationdate
#           from users u
#          where u.id >1
#         ) x
#         left join posts p
#           on x.usr_id = p.owneruserid
#          and p.posttypeid in( '1', '2')
# ;


In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select yeardiff, count(q_id) as cnt \
  from public.tt_year_from_signup_a \
 group by yeardiff \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_year_from_signup_a = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'cnt' 
])


In [ ]:
# 전체 데이터셋 
with open('../../data/df_year_from_signup_a.pkl', 'wb') as f:
    pickle.dump(df_year_from_signup_a, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select yeardiff, count(q_id) as cnt \
  from public.tt_year_from_signup_qna \
 group by yeardiff \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_year_from_signup_qna = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'cnt' 
])


In [ ]:
# 전체 데이터셋 
with open('../../data/df_year_from_signup_qna.pkl', 'wb') as f:
    pickle.dump(df_year_from_signup_qna, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select yeardiff, count(q_id) as cnt \
  from public.tt_year_from_signup_q \
 group by yeardiff \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_year_from_signup_q = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'cnt' 
])


In [ ]:
# 전체 데이터셋 
with open('../../data/df_year_from_signup_q.pkl', 'wb') as f:
    pickle.dump(df_year_from_signup_q, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.yeardiff, avg(x.cnt) as mean_cnt , median(x.cnt) as med_cnt \
  from ( \
select a.yeardiff, a.usr_id, count(a.q_id) as cnt \
  from public.tt_year_from_signup_qna a \
 where a.yeardiff >=0 \
 group by a.yeardiff, a.usr_id \
  ) x \
 group by x.yeardiff \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_year_from_signup_qna_mm = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'mean_cnt' 
, 'med_cnt'
])


In [ ]:
# 전체 데이터셋 
with open('../../data/df_year_from_signup_qna_mm.pkl', 'wb') as f:
    pickle.dump(df_year_from_signup_qna_mm, f)

In [ ]:
# create table tt_year_from_signup_qna as
# select x.usr_id
#      , p.id as q_id
#      , ceil(DATE_PART('day',p.creationdate::timestamp-x.usr_creationdate::timestamp)/365) as yeardiff
#      , p.creationdate, x.usr_creationdate
#   from (
#         select u.id as usr_id, u.creationdate as usr_creationdate
#           from users u
#          where u.id >1
#         ) x
#         left join posts p
#           on x.usr_id = p.owneruserid
#          and p.posttypeid in( '1', '2')
# ;


In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.yeardiff, avg(x.cnt) as mean_cnt , median(x.cnt) as med_cnt \
  from ( \
select a.yeardiff, a.usr_id, count(a.q_id) as cnt \
  from public.tt_year_from_signup_q a \
 where a.yeardiff >=0 \
 group by a.yeardiff, a.usr_id \
  ) x \
 group by x.yeardiff \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_year_from_signup_q_mm = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'mean_cnt' 
, 'med_cnt'
])


In [ ]:
# 전체 데이터셋 
with open('../../data/df_year_from_signup_q_mm.pkl', 'wb') as f:
    pickle.dump(df_year_from_signup_q_mm, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.yeardiff, avg(x.cnt) as mean_cnt , median(x.cnt) as med_cnt \
  from ( \
select a.yeardiff, a.usr_id, count(a.q_id) as cnt \
  from public.tt_year_from_signup_a a \
 where a.yeardiff >=0 \
 group by a.yeardiff, a.usr_id \
  ) x \
 group by x.yeardiff \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_year_from_signup_a_mm = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'mean_cnt' 
, 'med_cnt'
])


In [ ]:
# 전체 데이터셋 
with open('../../data/df_year_from_signup_a_mm.pkl', 'wb') as f:
    pickle.dump(df_year_from_signup_a_mm, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.yeardiff, x.c_gpt_yn, x.u_c_gpt_yn, count(distinct x.q_id) as q_cnt, count(distinct x.usr_id) as usr_cnt \
  from ( \
		select a.yeardiff \
			, case when to_char(a.creationdate, 'yyyy-mm-dd') >= '2022-11-30' then 1 else 0 end as c_gpt_yn \
			, case when to_char(a.usr_creationdate, 'yyyy-mm-dd') >= '2022-11-30' then 1 else 0 end as u_c_gpt_yn \
			, a.q_id \
			, a.usr_id \
		  from public.tt_year_from_signup_qna a \
  ) x \
group by x.yeardiff, x.c_gpt_yn, x.u_c_gpt_yn \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_year_gpt_yn = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'c_gpt_yn' 
, 'u_c_gpt_yn'
, 'q_cnt'
, 'usr_cnt'
])


In [ ]:
# 전체 데이터셋 
with open('../../data/df_year_gpt_yn.pkl', 'wb') as f:
    pickle.dump(df_year_gpt_yn, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.yeardiff, x.c_gpt_yn, x.u_c_gpt_yn, count(distinct x.q_id) as q_cnt, count(distinct x.usr_id) as usr_cnt \
  from ( \
		select a.yeardiff \
			, case when to_char(a.creationdate, 'yyyy-mm-dd') >= '2022-11-30' then 1 else 0 end as c_gpt_yn \
			, case when to_char(a.usr_creationdate, 'yyyy-mm-dd') >= '2022-11-30' then 1 else 0 end as u_c_gpt_yn \
			, a.q_id \
			, a.usr_id \
		  from public.tt_year_from_signup_a a \
  ) x \
group by x.yeardiff, x.c_gpt_yn, x.u_c_gpt_yn \
 ;"
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_year_gpt_yn_a = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'c_gpt_yn' 
, 'u_c_gpt_yn'
, 'q_cnt'
, 'usr_cnt'
])


In [ ]:
# 전체 데이터셋 
with open('../../data/df_year_gpt_yn_a.pkl', 'wb') as f:
    pickle.dump(df_year_gpt_yn_a, f)

In [ ]:
# select xx.usr_creationyear, xx.postcreationyear, avg(post_cnt) as avg_post_cnt, median(post_cnt) as med_post_cnt
#   from (
#         select x.usr_creationyear, x.usr_id, x.postcreationyear, sum(post_cnt) as post_cnt
#           from (select a.usr_id, a.postcreationyear, a.posttypeid,a.post_cnt, b.creationyear as usr_creationyear
#           from tt_year_per_post_cnt_01 a
#              , users b
#         where a.usr_id = b.id
#               )x
#         group by x.usr_creationyear, x.usr_id, x.postcreationyear
       
#            ) xx 
# group by xx.usr_creationyear, xx.postcreationyear
# ;

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select xx.usr_creationyear, xx.postcreationyear, avg(post_cnt) as avg_post_cnt, median(post_cnt) as med_post_cnt \
  from ( \
        select x.usr_creationyear, x.usr_id, x.postcreationyear, sum(post_cnt) as post_cnt \
          from (select a.usr_id, a.postcreationyear, a.posttypeid,a.post_cnt, b.creationyear as usr_creationyear \
          from tt_year_per_post_cnt_01 a \
             , users b \
        where a.usr_id = b.id \
              )x \
        group by x.usr_creationyear, x.usr_id, x.postcreationyear \
           ) xx \
group by xx.usr_creationyear, xx.postcreationyear \
; \
" 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df = pd.DataFrame(rows, columns = [
  'usr_creationyear' 
, 'post_creationyear'
, 'avg_post_cnt' 
, 'med_post_cnt' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_usr_post_cre_avg_med.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select xx.usr_creationyear, xx.postcreationyear, avg(post_cnt) as avg_post_cnt, median(post_cnt) as med_post_cnt \
  from ( \
        select x.usr_creationyear, x.usr_id, x.postcreationyear, sum(post_cnt) as post_cnt \
          from (select a.usr_id, a.postcreationyear, a.posttypeid,a.post_cnt, b.creationyear as usr_creationyear \
          from tt_year_per_post_cnt_01 a \
             , users b \
        where a.usr_id = b.id \
          and a.posttypeid = '1' \
              )x \
        group by x.usr_creationyear, x.usr_id, x.postcreationyear \
           ) xx \
group by xx.usr_creationyear, xx.postcreationyear \
; \
" 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df = pd.DataFrame(rows, columns = [
  'usr_creationyear' 
, 'q_creationyear'
, 'avg_post_cnt' 
, 'med_post_cnt' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_usr_q_cre_avg_med.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select xx.usr_creationyear, xx.postcreationyear, avg(post_cnt) as avg_post_cnt, median(post_cnt) as med_post_cnt \
  from ( \
        select x.usr_creationyear, x.usr_id, x.postcreationyear, sum(post_cnt) as post_cnt \
          from (select a.usr_id, a.postcreationyear, a.posttypeid,a.post_cnt, b.creationyear as usr_creationyear \
          from tt_year_per_post_cnt_01 a \
             , users b \
        where a.usr_id = b.id \
          and a.posttypeid = '2' \
              )x \
        group by x.usr_creationyear, x.usr_id, x.postcreationyear \
           ) xx \
group by xx.usr_creationyear, xx.postcreationyear \
; \
" 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df = pd.DataFrame(rows, columns = [
  'usr_creationyear' 
, 'a_creationyear'
, 'avg_post_cnt' 
, 'med_post_cnt' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_usr_a_cre_avg_med.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.usr_creationyear,x.posttypeid, x.postcreationyear, sum(post_cnt) as post_cnt \
  from (select a.usr_id, a.postcreationyear, a.posttypeid,a.post_cnt, b.creationyear as usr_creationyear \
  from tt_year_per_post_cnt_01 a \
     , users b \
where a.usr_id = b.id \
      )x \
group by x.usr_creationyear,x.posttypeid, x.postcreationyear \
; \
" 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df = pd.DataFrame(rows, columns = [
  'usr_creationyear' 
, 'posttypeid'
, 'postcreationyear' 
, 'post_cnt' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_usr_post_cre_cnt.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.yeardiff, x.usr_creationyear, avg(x.cnt) as avg_cnt, median(x.cnt) as med_cnt \
  from ( \
        select a.yeardiff, a.usr_id, b.creationyear as usr_creationyear, count(a.q_id) as cnt \
          from public.tt_year_from_signup_qna a \
             , users b \
        where a.usr_id = b.id \
          and a.yeardiff >=0 \
         group by a.yeardiff, a.usr_id, b.creationyear \
        ) x \
group by x.yeardiff, x.usr_creationyear \
; \
" 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'usr_creationyear'
, 'avg_cnt' 
, 'med_cnt' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_diff_usrcre_avg_med.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select xx.usr_creationyear, xx.postcreationyear, count(xx.usr_id) as usr_cnt , sum(post_yn) post_y_usr_cnt \
 from ( select x.usr_creationyear, x.usr_id, x.postcreationyear, case when sum(post_cnt) =0 then 1 else 0 end as post_yn \
          from (select a.usr_id, a.postcreationyear, a.posttypeid,a.post_cnt, b.creationyear as usr_creationyear \
          from tt_year_per_post_cnt_01 a \
             , users b \
        where a.usr_id = b.id \
              )x \
        group by x.usr_creationyear, x.usr_id, x.postcreationyear \
    ) xx \
group by xx.usr_creationyear, xx.postcreationyear \
; \
" 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df = pd.DataFrame(rows, columns = [
  'usr_creationyear' 
, 'postcreationyear' 
, 'usr_cnt'
, 'post_y_usr_cnt'
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_no_post_per_year.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select tot.yeardiff, tot.usr_cnt as tot_usr_cnt, qq.usr_cnt as q_usr_cnt, aa.usr_cnt as a_usr_cnt \
from (select a.yeardiff, count(distinct a.usr_id) as usr_cnt \
  from public.tt_year_from_signup_qna a \
group by a.yeardiff \
order by a.yeardiff) tot, \
(select a.yeardiff, count(distinct a.usr_id) as usr_cnt \
  from public.tt_year_from_signup_q a \
group by a.yeardiff \
order by a.yeardiff) qq, \
(select a.yeardiff, count(distinct a.usr_id) as usr_cnt \
  from public.tt_year_from_signup_a a \
group by a.yeardiff \
order by a.yeardiff) aa \
where tot.yeardiff = qq.yeardiff \
  and tot.yeardiff = aa.yeardiff \
; \
" 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df = pd.DataFrame(rows, columns = [
  'yeardiff' 
, 'tot_usr_cnt' 
, 'q_usr_cnt'
, 'a_usr_cnt'
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_year_from_signup_cnt.pkl', 'wb') as f:
    pickle.dump(df, f)

In [13]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select creationyear, posttypeid, count(distinct owneruserid) as usr_cnt \
  from posts \
 where posttypeid in ('1', '2') \
 group by creationyear, posttypeid \
; \
" 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [14]:
df = pd.DataFrame(rows, columns = [
  'creationyear' 
, 'posttypeid' 
, 'usr_cnt'
])

In [15]:
# 전체 데이터셋 
with open('../../data/df_active_usr_per_year.pkl', 'wb') as f:
    pickle.dump(df, f)

In [16]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
 select x. creationdate, x. posttypeid, count(distinct x.owneruserid) as usr_cnt \
   from ( \
   select to_char(creationdate, 'yyyy-mm-dd') as creationdate , posttypeid, owneruserid \
  from posts \
 where posttypeid in ('1', '2') \
   ) x \
 group by x. creationdate, x. posttypeid \
; \
" 
)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()


In [17]:
df = pd.DataFrame(rows, columns = [
  'creationdate' 
, 'posttypeid' 
, 'usr_cnt'
])

In [18]:
# 전체 데이터셋 
with open('../../data/df_active_usr_per_day.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.vote_creationdate, count(id) as cnt \
    from ( \
    select to_char(creationdate, 'yyyy-mm-dd') as vote_creationdate, id \
      from votes \
    ) x \
  group by x.vote_creationdate \
     ; \
" 
)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()


In [ ]:
df = pd.DataFrame(rows, columns = [
  'vote_creationdate' 
, 'cnt' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_vote_cnt_per_day.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select post_creationdate, count(vote_id) as cnt \
   from (select to_char(a.creationdate, 'yyyy-mm-dd') post_creationdate, b.id as vote_id \
   from posts a \
      , votes b \
     where a.id = b.postid \
       and b.creationdate - a.creationdate <= '30 days' \
       and a.posttypeid in ('1', '2') \
 ) x \
 group by post_creationdate \
     ; \
" 
)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()


In [ ]:
df = pd.DataFrame(rows, columns = [
  'post_creationdate' 
, 'cnt' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_vote_cnt_within_1month_per_day.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select post_creationdate, count(vote_id) as cnt \
   from (select to_char(a.creationdate, 'yyyy-mm-dd') post_creationdate, b.id as vote_id \
   from posts a \
      , votes b \
     where a.id = b.postid \
       and b.creationdate - a.creationdate <= '30 days' \
       and a.posttypeid in ('1', '2') \
       and b.votetypeid in ('2', '3') \
 ) x \
 group by post_creationdate \
     ; \
" 
)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()


In [ ]:
df = pd.DataFrame(rows, columns = [
  'post_creationdate' 
, 'cnt' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_vote_cnt_within_1month_per_day_qna.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.post_creationdate, x.posttypeid, count(x.vote_id) as cnt \
  from (select to_char(a.creationdate, 'yyyy-mm-dd') post_creationdate, b.id as vote_id, a.posttypeid \
  from posts a \
    , votes b \
    where a.id = b.postid \
      and b.creationdate - a.creationdate <= '30 days' \
      and a.posttypeid in ('1', '2') \
      and b.votetypeid in ('2', '3') \
) x \
group by x.post_creationdate, x.posttypeid \
; \
" 
)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()


In [ ]:
df = pd.DataFrame(rows, columns = [
  'post_creationdate' 
, 'posttypeid' 
, 'vote_cnt'
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_vote_cnt_within_1month_per_daytype_qna.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select date, posttypeid, count(*) as cnt \
  from (select to_char(creationdate, 'yyyy-mm-dd') as date, posttypeid , id \
  from posts \
) x \
group by date, posttypeid \
  ; \
" 
)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()


In [ ]:
df = pd.DataFrame(rows, columns = [
  'post_creationdate' 
, 'posttypeid' 
, 'vote_cnt'
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_vote_cnt_within_1month_per_daytype_qna.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
# 파일 불러오기
with open('../../data/df_body.pkl', 'rb') as f:
    df_body = pickle.load(f)


In [ ]:
df_body['date'] = df_body['creationdate'].dt.date
df_body['year'] = df_body['creationdate'].dt.year
df_body['month'] = df_body['creationdate'].dt.month
df_body['day'] = df_body['creationdate'].dt.day

In [ ]:
df_body1 = df_body[(df_body['creationdate']>='2022-10-01') & (df_body['creationdate']<='2022-10-30')]
df_body2 = df_body[(df_body['creationdate']>='2021-10-01') & (df_body['creationdate']<='2021-10-30')]
df_body3 = df_body[(df_body['creationdate']>='2020-10-01') & (df_body['creationdate']<='2020-10-30')]
df_body4 = df_body[(df_body['creationdate']>='2019-10-01') & (df_body['creationdate']<='2019-10-30')]
df_body5 = df_body[(df_body['creationdate']>='2018-10-01') & (df_body['creationdate']<='2018-10-30')]
df_body6 = df_body[(df_body['creationdate']>='2023-10-01') & (df_body['creationdate']<='2023-10-30')]

In [ ]:
df_bodyconcat = pd.concat([df_body1, df_body2, df_body3, df_body4, df_body5, df_body6], axis = 0)

In [ ]:
df_bodyconcat.shape

In [ ]:
df_body = df_body[df_body['creationdate']>='2021-10-01']

In [ ]:
df_body['creationdate'].min()

## 2. Preprocessing 

In [ ]:
def cleanhtml(raw_html):
  #1. replace newline to space
  # To analyze the text, it would be hard to understand all source code in python, so replace all the <code> tag first
  # src_re = re.compile('<code>(?=[^<]*\n)[\s\S]*?<\/code>')
  # src_re = re.compile('<code>[^<]*\\n[^<]*<\/code>')
  src_re = re.compile('<code>(.*?)<\/code>')
  cleantext_1 = re.sub(src_re, '', raw_html.replace('\n', '_**_'))
  cleantext_1 = cleantext_1.replace('_**_', '\n')

  tag_re = re.compile('<.*?>')
  cleantext_2 = re.sub(tag_re, '', cleantext_1)

  cleantext_2 = re.sub(r"\n", "", cleantext_2)
  cleantext_2 = re.sub(r"\r", "", cleantext_2)
  cleantext_2 = re.sub(r"\t", "", cleantext_2)
  cleantext_2 = ''.join([x for x in cleantext_2 if x not in string.punctuation])
  return cleantext_2

In [ ]:
# apply the function, cleanhtml to the question and body text
df_body['body_prep'] = df_body['body'].apply(cleanhtml)

In [ ]:
df_bodyconcat['body_prep'] = df_bodyconcat['body'].apply(cleanhtml)

In [ ]:
df_wl = df_body[['id', 'creationdate', 'body_prep', 'posttypeid']].copy()

In [ ]:
df_bodyconcat_wl = df_bodyconcat[['id', 'creationdate', 'body_prep']].copy()

In [ ]:
def wordlen(text):
    return [x for x in text.split(' ') if len(x) >= 4]

In [ ]:
df_wl['over4'] = df_wl['body_prep'].apply(wordlen)

In [ ]:
df_bodyconcat_wl['over4'] = df_bodyconcat_wl['body_prep'].apply(wordlen)

In [ ]:
# 전체 데이터셋 
with open('../../data/df_wl.pkl', 'wb') as f:
    pickle.dump(df_wl, f)

In [ ]:
# 전체 데이터셋 
with open('../../data/df_bodyconcat_wl.pkl', 'wb') as f:
    pickle.dump(df_bodyconcat_wl, f)

In [ ]:
df_body[df_body['id'] == 73954716]

In [ ]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select \
	x.cdate , \
	sum(x.python_yn) as python, \
	sum(x.javascript_yn) as javascript, \
	sum(x.reactjs_yn) as reactjs, \
	sum(x.java_yn) as java, \
	sum(x.csharp_yn) as csharp, \
	sum(x.html_yn) as html, \
	sum(x.r_yn) as r, \
	sum(x.pandas_yn) as pandas, \
	sum(x.css_yn) as css, \
	sum(x.flutter_yn) as flutter, \
	sum(x.sql_yn) as sql, \
	sum(x.android_yn) as android, \
	sum(x.nodejs_yn) as nodejs, \
	sum(x.cplusplus_yn) as cplusplus, \
	sum(x.typescript_yn) as typescript, \
	sum(x.php_yn) as php, \
	sum(x.python_3x_yn) as python_3x, \
	sum(x.dataframe_yn) as dataframe, \
	sum(x.arrays_yn) as arrays, \
	sum(x.angular_yn) as angular, \
	sum(x.django_yn) as django, \
	sum(x.json_yn) as json, \
	sum(x.dart_yn) as dart, \
	sum(x.mysql_yn) as mysql, \
	sum(x.c_yn) as c, \
	sum(x.excel_yn) as excel, \
	sum(x.laravel_yn) as laravel, \
	sum(x.swift_yn) as swift, \
	sum(x.spring_boot_yn) as spring_boot, \
	sum(x.amazon_web_services_yn) as amazon_web_services, \
	sum(x.react_native_yn) as react_native, \
	sum(x.postgresql_yn) as postgresql, \
	sum(x.azure_yn) as azure, \
	sum(x.kotlin_yn) as kotlin, \
	sum(x.docker_yn) as docker, \
	sum(x.list_yn) as list, \
	sum(x.firebase_yn) as firebase, \
	sum(x.ios_yn) as ios, \
	sum(x.sql_server_yn) as sql_server, \
	sum(x.jquery_yn) as jquery, \
	sum(x.mongodb_yn) as mongodb, \
	sum(x.vuejs_yn) as vuejs, \
	sum(x.numpy_yn) as numpy, \
	sum(x.net_yn) as net, \
	sum(x.vba_yn) as vba, \
	sum(x.regex_yn) as regex, \
	sum(x.string_yn) as string, \
	sum(x.spring_yn) as spring, \
	sum(x.powershell_yn) as powershell, \
	sum(x.linux_yn) as linux \
from tt_postnum_per_tag x \
group by x.cdate \
	; \
" 
)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()



In [ ]:
df = pd.DataFrame(rows, columns = [
'cdate', 'python', 'javascript', 'reactjs', 'java', 'c#',
       'html', 'r', 'pandas', 'css', 'flutter', 'sql', 'android', 'nodejs',
       'cplusplus', 'typescript', 'php', 'python_3x', 'dataframe', 'arrays',
       'angular', 'django', 'json', 'dart', 'mysql', 'c', 'excel', 'laravel',
       'swift', 'spring_boot', 'amazon_web_services', 'react_native',
       'postgresql', 'azure', 'kotlin', 'docker', 'list', 'firebase', 'ios',
       'sql_server', 'jquery', 'mongodb', 'vuejs', 'numpy', 'net', 'vba',
       'regex', 'string', 'spring', 'powershell', 'linux'
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_postcnt_per_tag.pkl', 'wb') as f:
    pickle.dump(df, f)

In [3]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.year, \
       sum(x.python)              as python, \
       sum(x.javascript)          as javascript, \
       sum(x.reactjs)             as reactjs, \
       sum(x.java)                as java, \
       sum(x.csharp)              as csharp, \
       sum(x.html)                as html, \
       sum(x.r)                   as r, \
       sum(x.pandas)              as pandas, \
       sum(x.css)                 as css, \
       sum(x.flutter)             as flutter, \
       sum(x.sql)                 as sql, \
       sum(x.android)             as android, \
       sum(x.nodejs)              as nodejs, \
       sum(x.cplusplus)           as cplusplus, \
       sum(x.typescript)          as typescript, \
       sum(x.php)                 as php, \
       sum(x.python_3x)           as python_3x, \
       sum(x.dataframe)           as dataframe, \
       sum(x.arrays)              as arrays, \
       sum(x.angular)             as angular, \
       sum(x.django)              as django, \
       sum(x.json)                as json, \
       sum(x.dart)                as dart, \
       sum(x.mysql)               as mysql, \
       sum(x.c)                   as c, \
       sum(x.excel)               as excel, \
       sum(x.laravel)             as laravel, \
       sum(x.swift)               as swift, \
       sum(x.spring_boot)         as spring_boot, \
       sum(x.amazon_web_services) as amazon_web_services, \
       sum(x.react_native)        as react_native, \
       sum(x.postgresql)          as postgresql, \
       sum(x.azure)               as azure, \
       sum(x.kotlin)              as kotlin, \
       sum(x.docker)              as docker, \
       sum(x.list)                as list, \
       sum(x.firebase)            as firebase, \
       sum(x.ios)                 as ios, \
       sum(x.sql_server)          as sql_server, \
       sum(x.jquery)              as jquery, \
       sum(x.mongodb)             as mongodb, \
       sum(x.vuejs)               as vuejs, \
       sum(x.numpy)               as numpy, \
       sum(x.net)                 as net, \
       sum(x.vba)                 as vba, \
       sum(x.regex)               as regex, \
       sum(x.string)              as string, \
       sum(x.spring)              as spring, \
       sum(x.powershell)          as powershell, \
       sum(x.linux)               as linux \
from ( \
         select SUBSTRING(cdate, 1, 4) as year, \
                python_yn              as python, \
                javascript_yn          as javascript, \
                reactjs_yn             as reactjs, \
                java_yn                as java, \
                csharp_yn                as csharp, \
                html_yn as html, \
                r_yn                   as r, \
                pandas_yn              as pandas, \
                css_yn                 as css, \
                flutter_yn             as flutter, \
                sql_yn                 as sql, \
                android_yn             as android, \
                nodejs_yn              as nodejs, \
                cplusplus_yn           as cplusplus, \
                typescript_yn          as typescript, \
                php_yn                 as php, \
                python_3x_yn           as python_3x, \
                dataframe_yn           as dataframe, \
                arrays_yn              as arrays, \
                angular_yn             as angular, \
                django_yn              as django, \
                json_yn                as json, \
                dart_yn                as dart, \
                mysql_yn               as mysql, \
                c_yn                   as c, \
                excel_yn               as excel, \
                laravel_yn             as laravel, \
                swift_yn               as swift, \
                spring_boot_yn         as spring_boot, \
                amazon_web_services_yn as amazon_web_services, \
                react_native_yn        as react_native, \
                postgresql_yn          as postgresql, \
                azure_yn               as azure, \
                kotlin_yn              as kotlin, \
                docker_yn              as docker, \
                list_yn                as list, \
                firebase_yn            as firebase, \
                ios_yn                 as ios, \
                sql_server_yn          as sql_server, \
                jquery_yn              as jquery, \
                mongodb_yn             as mongodb, \
                vuejs_yn               as vuejs, \
                numpy_yn               as numpy, \
                net_yn                 as net, \
                vba_yn                 as vba, \
                regex_yn               as regex, \
                string_yn              as string, \
                spring_yn              as spring, \
                powershell_yn          as powershell, \
                linux_yn               as linux \
         from tt_tot_postnum_per_tag \
     ) x \
group by x.year \
; \
" 
)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()



In [4]:
df = pd.DataFrame(rows, columns = [
'year', 'python', 'javascript', 'reactjs', 'java', 'c#',
       'html', 'r', 'pandas', 'css', 'flutter', 'sql', 'android', 'nodejs',
       'cplusplus', 'typescript', 'php', 'python_3x', 'dataframe', 'arrays',
       'angular', 'django', 'json', 'dart', 'mysql', 'c', 'excel', 'laravel',
       'swift', 'spring_boot', 'amazon_web_services', 'react_native',
       'postgresql', 'azure', 'kotlin', 'docker', 'list', 'firebase', 'ios',
       'sql_server', 'jquery', 'mongodb', 'vuejs', 'numpy', 'net', 'vba',
       'regex', 'string', 'spring', 'powershell', 'linux'
])

In [5]:
# 전체 데이터셋 
with open('../../data/df_postcnt_per_tag_year.pkl', 'wb') as f:
    pickle.dump(df, f)

In [7]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select x.year, \
       sum(x.python)              as python, \
       sum(x.javascript)          as javascript, \
       sum(x.reactjs)             as reactjs, \
       sum(x.java)                as java, \
       sum(x.csharp)              as csharp, \
       sum(x.html)                as html, \
       sum(x.r)                   as r, \
       sum(x.pandas)              as pandas, \
       sum(x.css)                 as css, \
       sum(x.flutter)             as flutter, \
       sum(x.sql)                 as sql, \
       sum(x.android)             as android, \
       sum(x.nodejs)              as nodejs, \
       sum(x.cplusplus)           as cplusplus, \
       sum(x.typescript)          as typescript, \
       sum(x.php)                 as php, \
       sum(x.python_3x)           as python_3x, \
       sum(x.dataframe)           as dataframe, \
       sum(x.arrays)              as arrays, \
       sum(x.angular)             as angular, \
       sum(x.django)              as django, \
       sum(x.json)                as json, \
       sum(x.dart)                as dart, \
       sum(x.mysql)               as mysql, \
       sum(x.c)                   as c, \
       sum(x.excel)               as excel, \
       sum(x.laravel)             as laravel, \
       sum(x.swift)               as swift, \
       sum(x.spring_boot)         as spring_boot, \
       sum(x.amazon_web_services) as amazon_web_services, \
       sum(x.react_native)        as react_native, \
       sum(x.postgresql)          as postgresql, \
       sum(x.azure)               as azure, \
       sum(x.kotlin)              as kotlin, \
       sum(x.docker)              as docker, \
       sum(x.list)                as list, \
       sum(x.firebase)            as firebase, \
       sum(x.ios)                 as ios, \
       sum(x.sql_server)          as sql_server, \
       sum(x.jquery)              as jquery, \
       sum(x.mongodb)             as mongodb, \
       sum(x.vuejs)               as vuejs, \
       sum(x.numpy)               as numpy, \
       sum(x.net)                 as net, \
       sum(x.vba)                 as vba, \
       sum(x.regex)               as regex, \
       sum(x.string)              as string, \
       sum(x.spring)              as spring, \
       sum(x.powershell)          as powershell, \
       sum(x.linux)               as linux \
from ( \
         select SUBSTRING(cdate, 1, 4) as year, \
                python              as python, \
                javascript          as javascript, \
                reactjs             as reactjs, \
                java                as java, \
                csharp                as csharp, \
                html as html, \
                r                   as r, \
                pandas              as pandas, \
                css                 as css, \
                flutter             as flutter, \
                sql                 as sql, \
                android             as android, \
                nodejs              as nodejs, \
                cplusplus           as cplusplus, \
                typescript          as typescript, \
                php                 as php, \
                python_3x           as python_3x, \
                dataframe           as dataframe, \
                arrays              as arrays, \
                angular             as angular, \
                django              as django, \
                json                as json, \
                dart                as dart, \
                mysql               as mysql, \
                c                   as c, \
                excel               as excel, \
                laravel             as laravel, \
                swift               as swift, \
                spring_boot         as spring_boot, \
                amazon_web_services as amazon_web_services, \
                react_native        as react_native, \
                postgresql          as postgresql, \
                azure               as azure, \
                kotlin              as kotlin, \
                docker              as docker, \
                list                as list, \
                firebase            as firebase, \
                ios                 as ios, \
                sql_server          as sql_server, \
                jquery              as jquery, \
                mongodb             as mongodb, \
                vuejs               as vuejs, \
                numpy               as numpy, \
                net                 as net, \
                vba                 as vba, \
                regex               as regex, \
                string              as string, \
                spring              as spring, \
                powershell          as powershell, \
                linux               as linux \
         from tt_tot_postnum_per_tag_01 \
     ) x \
group by x.year \
; \
" 
)

    rows = cur.fetchall()
    
except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()



In [10]:
df = pd.DataFrame(rows, columns = [
'year', 'python', 'javascript', 'reactjs', 'java', 'c#',
       'html', 'r', 'pandas', 'css', 'flutter', 'sql', 'android', 'nodejs',
       'cplusplus', 'typescript', 'php', 'python_3x', 'dataframe', 'arrays',
       'angular', 'django', 'json', 'dart', 'mysql', 'c', 'excel', 'laravel',
       'swift', 'spring_boot', 'amazon_web_services', 'react_native',
       'postgresql', 'azure', 'kotlin', 'docker', 'list', 'firebase', 'ios',
       'sql_server', 'jquery', 'mongodb', 'vuejs', 'numpy', 'net', 'vba',
       'regex', 'string', 'spring', 'powershell', 'linux'
])

In [11]:
# 전체 데이터셋 
with open('../../data/df_postcnt_per_tag_year_01.pkl', 'wb') as f:
    pickle.dump(df, f)

In [13]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
  select xx.cdate, xx.posttypeid, avg(q_cnt) as avg_q_cnt, median(q_cnt) as med_q_cnt \
    from (select x.usr_id, x.cdate, x.posttypeid, count(x.q_id) as q_cnt \
  			from ( \
				select a.id as usr_id, b.posttypeid, to_char(b.creationdate, 'yyyy-mm-dd' ) as cdate, b.id as q_id \
				  from users a \
				     , posts b \
				 where a.id = b.owneruserid \
				   and b.posttypeid in ('1', '2') \
				) x \
			 group by x.usr_id, x.cdate, x.posttypeid \
			) xx \
	group by xx.cdate, xx.posttypeid \
; \
" 
)

    rows = cur.fetchall()
    
except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()



In [14]:
df = pd.DataFrame(rows, columns = [
'cdate', 'posttypeid', 'avg_q_cnt', 'med_q_cnt'
])

In [15]:
# 전체 데이터셋 
with open('../../data/df_uuser_per_post.pkl', 'wb') as f:
    pickle.dump(df, f)

In [3]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select postcreationyear, posttypeid, post_cnt, count(*) as value \
  from tt_year_per_post_cnt_01 \
  where post_cnt != 0 \
  group by postcreationyear, posttypeid, post_cnt \
  ; \
; \
" 
    )
    rows = cur.fetchall()
    
except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()



In [4]:
df = pd.DataFrame(rows, columns = [
'postcreationyear', 'posttypeid', 'post_cnt', 'value'
])

In [5]:
# 전체 데이터셋 
with open('../../data/df_uuser_per_year.pkl', 'wb') as f:
    pickle.dump(df, f)

In [6]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select xx.cdate, count(xx.id) as cnt, 300 as sample_num, 300/count(xx.id)*100 as rate \
  from ( \
		select id, to_char(creationdate, 'yyyy-mm-dd') as cdate \
		  from posts \
		 where posttypeid ='1' \
		   and tags like '%<python>%' \
		   and creationdate between '2021-11-30' and '2023-11-30' \
  		) xx, \
  		( \
  		select distinct to_char(creationdate, 'yyyy-mm-dd') as cdate \
		  from tt_posts_difficulty_target \
		 where ver >= 50000 \
  		) yy \
 where xx.cdate = yy.cdate \
 group by xx.cdate \
; \
" 
    )
    rows = cur.fetchall()
    
except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()


In [7]:
df = pd.DataFrame(rows, columns = [
'cdate', 'cnt', 'sample_num', 'rate'
])

In [8]:
# 전체 데이터셋 
with open('../../data/df_sample_rate.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:

conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select xx.cdate, xx.posttypeid,xx.qna_cnt, count(xx.usr_id) as value \
    from (select x.usr_id, x.cdate, x.posttypeid, count(x.qna_id) as qna_cnt \
  			from ( \
				select a.id as usr_id, b.posttypeid, to_char(b.creationdate, 'yyyy-mm' ) as cdate, b.id as qna_id \
				  from users a \
				     , posts b \
				 where a.id = b.owneruserid \
				   and b.posttypeid in ('1', '2') \
				   and b.creationdate >='2021-01-01' \
				) x \
			 group by x.usr_id, x.cdate, x.posttypeid \
			) xx \
	group by xx.cdate, xx.posttypeid, xx.qna_cnt \
; \
" 
    )
    rows = cur.fetchall()
    
except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()


In [ ]:
df = pd.DataFrame(rows, columns = [
'cdate', 'posttypeid', 'qna_cnt', 'value'
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_qcnt_value_per_ymon.pkl', 'wb') as f:
    pickle.dump(df, f)